<font size = "5"><b>ПРОЕКТНАЯ РАБОТА</b></font><br/>
<font size = "4"><b>SQL - Сервис для чтения книг</b></font>

<a name = "0.0"></a><font size = "4"><b>Содержание</b></font><br/>
* [0. Описание проекта](#0.)
* [1. Ознакомление и предобработка первичных данных](#1.)
* [2. Задание № 1: Посчитать, сколько книг вышло после 1 января 2000 года](#2.)
* [3. Задание № 2: Для каждой книги посчитать количество обзоров и среднюю оценку](#3.)
* [4. Задание № 3: Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры](#4.)
* [5. Задание № 4: Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками](#5.)
* [6. Задание № 5: Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#6.)
* [7. Вывод](#7.)

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задания**

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

_____________________

<a name = "0."></a><font size = "4"><b>Описание данных</b></font><br/>
<font size = "2">[содержание](#0.0)</font><br/>

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<a name = '1.'></a><font size = '4'><b>1. Ознакомление и предобработка первичных данных</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [138]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [139]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

In [140]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

Выведем все таблицы для ознаковления

In [141]:
# функция для вывода таблиц
def data(query):
    return pd.io.sql.read_sql(query, con = engine)

In [142]:
books = '''
SELECT *
FROM books
'''

aut = '''
SELECT *
FROM authors
'''

pub = '''
SELECT *
FROM publishers
'''

rat = '''
SELECT *
FROM ratings
'''

rev = '''
SELECT *
FROM reviews
'''

In [143]:
# Функция для вывода общей информации по dataframe
def base_info(df):
    print()
    print('------------------------------------Краткая сводка DataFrame------------------------------------')
    print()
    df.info()
    print()
    print('--------------------------------------Статистические данные-------------------------------------')
    print()
    display(df.describe())
    print()
    print('---------------------------------Пропущенные значения в столбцах--------------------------------')
    print()
    display(df.isna().sum())
    print()
    print('------------------------------Информация по дублирующимся строкам-------------------------------')
    print()
    display(df.duplicated().sum())
    print()
    print('------------------------------------------Общая таблица-----------------------------------------')
    print()
    display(df)
    print()

In [144]:
base_info(data(books))


------------------------------------Краткая сводка DataFrame------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB

--------------------------------------Статистические данные-------------------------------------



,book_id,author_id,num_pages,publisher_id
count,1000.000000,1000.000000,1000.00000,1000.000000
mean,500.500000,320.417000,389.11100,171.270000
std,288.819436,181.620172,229.39014,99.082685
min,1.000000,1.000000,14.00000,1.000000
25%,250.750000,162.750000,249.00000,83.000000
50%,500.500000,316.500000,352.00000,177.500000
75%,750.250000,481.000000,453.00000,258.000000
max,1000.000000,636.000000,2690.00000,340.000000



---------------------------------Пропущенные значения в столбцах--------------------------------



book_id             0
author_id           0
title               0
num_pages           0
publication_date    0
publisher_id        0
dtype: int64


------------------------------Информация по дублирующимся строкам-------------------------------



0


------------------------------------------Общая таблица-----------------------------------------



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


<b>Вывод:</b>
- датафрейм состоит имеет 1000 строк
- дата имеет тип object
- пропущенных значений нет
- Дублирующих строк нет
- В среднем 389 страниц в книге

In [145]:
base_info(data(aut))


------------------------------------Краткая сводка DataFrame------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB

--------------------------------------Статистические данные-------------------------------------



,author_id
count,636.000000
mean,318.500000
std,183.741666
min,1.000000
25%,159.750000
50%,318.500000
75%,477.250000
max,636.000000



---------------------------------Пропущенные значения в столбцах--------------------------------



author_id    0
author       0
dtype: int64


------------------------------Информация по дублирующимся строкам-------------------------------



0


------------------------------------------Общая таблица-----------------------------------------



,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


<b>Вывод:</b>
- датафрейм состоит имеет 636 строк
- пропущенных значений нет
- Дублирующих строк нет

In [146]:
base_info(data(pub))


------------------------------------Краткая сводка DataFrame------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB

--------------------------------------Статистические данные-------------------------------------



,publisher_id
count,340.000000
mean,170.500000
std,98.293777
min,1.000000
25%,85.750000
50%,170.500000
75%,255.250000
max,340.000000



---------------------------------Пропущенные значения в столбцах--------------------------------



publisher_id    0
publisher       0
dtype: int64


------------------------------Информация по дублирующимся строкам-------------------------------



0


------------------------------------------Общая таблица-----------------------------------------



,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


<b>Вывод:</b>
- датафрейм состоит имеет 340 строк
- пропущенных значений нет
- Дублирующих строк нет

In [147]:
base_info(data(rat))


------------------------------------Краткая сводка DataFrame------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB

--------------------------------------Статистические данные-------------------------------------



,rating_id,book_id,rating
count,6456.000000,6456.000000,6456.000000
mean,3228.500000,510.574195,3.928284
std,1863.831001,284.141636,0.943303
min,1.000000,1.000000,1.000000
25%,1614.750000,291.000000,3.000000
50%,3228.500000,506.000000,4.000000
75%,4842.250000,750.000000,5.000000
max,6456.000000,1000.000000,5.000000



---------------------------------Пропущенные значения в столбцах--------------------------------



rating_id    0
book_id      0
username     0
rating       0
dtype: int64


------------------------------Информация по дублирующимся строкам-------------------------------



0


------------------------------------------Общая таблица-----------------------------------------



,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


<b>Вывод:</b>
- датафрейм состоит имеет 6456 строк
- пропущенных значений нет
- Дублирующих строк нет
- Средний рейтинг 3.9, максимальный рейтинг 5

In [148]:
base_info(data(rev))


------------------------------------Краткая сводка DataFrame------------------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB

--------------------------------------Статистические данные-------------------------------------



,review_id,book_id
count,2793.000000,2793.000000
mean,1397.000000,504.693161
std,806.413976,288.472931
min,1.000000,1.000000
25%,699.000000,259.000000
50%,1397.000000,505.000000
75%,2095.000000,753.000000
max,2793.000000,1000.000000



---------------------------------Пропущенные значения в столбцах--------------------------------



review_id    0
book_id      0
username     0
text         0
dtype: int64


------------------------------Информация по дублирующимся строкам-------------------------------



0


------------------------------------------Общая таблица-----------------------------------------



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same enter true practice student system.
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy population. Pm vote take. Long born close thought person old. Visit ever school manage about.
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone foreign amount within cell. Ball poor hot that kid.


<b>Вывод:</b>
- датафрейм состоит имеет 2793 строк
- пропущенных значений нет
- Дублирующих строк нет

<a name = '2.'></a><font size = '4'><b>2. Задание № 1: Посчитать, сколько книг вышло после 1 января 2000 года</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [149]:
data(books).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [150]:
bcnt = '''
        SELECT COUNT(book_id) AS cnt
        FROM books
        WHERE publication_date > '2000-01-01';
'''

In [151]:
data(bcnt)

,cnt
0,819


- 819 книг вышло после 01 января 2020 года

<a name = '3.'></a><font size = '4'><b>3. Задание № 2: Для каждой книги посчитать количество обзоров и среднюю оценку</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [152]:
display(data(books).head())
display(data(rev).head())
display(data(rat).head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [153]:
cnt_rev_avg_rat = '''
                    SELECT b.book_id,
                            b.title,
                            b.author_id,
                            COUNT(DISTINCT rev.review_id) AS review_cnt,
                            ROUND(AVG(rat.rating), 2) AS rating_avg
                    FROM books AS b
                    LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
                    LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
                    GROUP BY b.book_id,
                            b.title,
                            b.author_id
                    ORDER BY review_cnt DESC, rating_avg DESC;
'''

In [154]:
pd.set_option('display.max_colwidth', None) # для отображения теста таблицы в полном объеме
data(cnt_rev_avg_rat)

,book_id,title,author_id,review_cnt,rating_avg
0,948,Twilight (Twilight #1),554,7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),236,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),236,6,4.29
3,656,The Book Thief,402,6,4.26
4,734,The Glass Castle,278,6,4.21
...,...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden Book),568,0,4.00
996,387,Leonardo's Notebooks,365,0,4.00
997,221,Essential Tales and Poems,151,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,174,0,3.67


- для каждой книги посчитали количество обзоров (больше всего 7 обзоров на книгу Twilight (Twilight #1) с рейтингом 3.66)
    * 6 обзоров с рейтингом 4.41 на книгу Harry Potter and the Prisoner of Azkaban (Harry Potter #3)
    * 6 обзоров с рейтингом 4.29 на книгу Harry Potter and the Chamber of Secrets (Harry Potter #2)

<a name = '4.'></a><font size = '4'><b>4. Задание № 3: Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [155]:
display(data(books).head())
display(data(pub).head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [156]:
book_cnt_pub = '''
                SELECT p.publisher_id,
                        p.publisher,
                        COUNT(b.book_id) AS book_cnt,
                        ROUND(AVG(b.num_pages), 0) AS pages_avg
                FROM books AS b
                JOIN publishers AS p ON b.publisher_id = p.publisher_id
                WHERE b.num_pages > 50
                GROUP BY p.publisher_id,
                        p.publisher
                ORDER BY book_cnt DESC
                LIMIT 1;
'''

In [157]:
data(book_cnt_pub)

,publisher_id,publisher,book_cnt,pages_avg
0,212,Penguin Books,42,380.0


- Издательство Penguin Books(212) выпустило 42 книги толще 50 страниц. Книги размером в среднем 380 страниц.

<a name = '5.'></a><font size = '4'><b>5. Задание № 4: Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [158]:
display(data(books).head())
display(data(aut).head())
display(data(rat).head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [159]:
author_high_rat = '''
                    WITH tab AS
                    (SELECT book_id,
                            COUNT(rating) AS rating_cnt,
                            AVG(rating) AS rating_avg
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating) >= 50)
                    
                    SELECT 
                            au.author,
                            au.author_id,
                            ROUND(AVG(rating_avg), 2) AS avg
                    FROM tab
                    JOIN books AS b ON tab.book_id = b.book_id
                    JOIN authors AS au ON b.author_id = au.author_id
                    GROUP BY
                            au.author,
                            au.author_id
                    ORDER BY avg DESC;
                    
                    
'''

In [160]:
data(author_high_rat)

,author,author_id,avg
0,J.K. Rowling/Mary GrandPré,236,4.28
1,J.R.R. Tolkien,240,4.26
2,Markus Zusak/Cao Xuân Việt Khương,402,4.26
3,Louisa May Alcott,376,4.19
4,Rick Riordan,498,4.08
5,William Golding,621,3.90
6,J.D. Salinger,235,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,469,3.79
8,William Shakespeare/Paul Werstine/Barbara A. Mowat,630,3.79
9,Lois Lowry,372,3.75


- Самую высокую оценку 4.28 книги (количество оценок более 49) имеет автор J.K. Rowling/Mary GrandPré

<a name = '6.'></a><font size = '4'><b>6. Задание № 5: Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

In [161]:
display(data(rev).head())
display(data(rat).head())

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [162]:
rev_rat = '''
            WITH t1 AS
            (SELECT username,
                    ROUND(COUNT(rating), 0) AS rating_avg
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50),
            
            t2 AS
            (SELECT rev.username,
                    COUNT(rev. text) AS cnt_text
            FROM reviews AS rev
            JOIN t1 ON rev.username = t1.username
            GROUP BY rev.username)
            
            SELECT ROUND(AVG(cnt_text), 2)
            FROM t2;
'''

In [163]:
data(rev_rat)

,round
0,24.33


- 24.33 среднее количество обзоров от пользователей, которые поставили больше 50 оценок

<a name = '7.'></a><font size = '4'><b>7. Вывод:</b></font><br>
<font size = '2'>[Содержание](#0.0)</font>

Ознакомились с базой данных, в которой информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

Задачи:
- Посчитайте, сколько книг вышло после 1 января 2000 года
    - 819 книг вышло после 01 января 2020 года
- Для каждой книги посчитайте количество обзоров и среднюю оценку
    - для каждой книги посчитали количество обзоров (больше всего 7 обзоров на книгу Twilight (Twilight #1) с рейтингом 3.66)
    * 6 обзоров с рейтингом 4.41 на книгу Harry Potter and the Prisoner of Azkaban (Harry Potter #3)
    * 6 обзоров с рейтингом 4.29 на книгу Harry Potter and the Chamber of Secrets (Harry Potter #2)
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры
    - Издательство Penguin Books(212) выпустило 42 книги толще 50 страниц. Книги размером в среднем 380 страниц
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками
    - Самую высокую оценку 4.28 книги (количество оценок более 49) имеет автор J.K. Rowling/Mary GrandPré
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок
    - 24.33 среднее количество обзоров от пользователей, которые поставили больше 50 оценок